# Automated ML

Import all dependencies.

In [1]:
import json
import azureml.core
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136394
aml-quickstarts-136394
southcentralus
3e42d11f-d64d-4173-af9b-12ecaa1030b3


## Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'automl-heart-experiment'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Create a Compute Cluster

In [4]:
amlcompute_cluster_name = "heart-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
For this project, I used the [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) from Kaggle. 

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

In [5]:
key = "heart-failure"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


### Explore

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration


Let me explain the reasoning behind my choices for the AutoML Config:
* **experiment_timeout_minutes**: I chose 15 minutes as the maximum amount of time the experiment can takee before it terminates because I have a small dataset with only 299 entries.
* **max_concurrent_iterations**: Represents the maximum number of iterations that would be executed in parallel. The default value is 1.
* **n_cross_validations**: To avoid overfitting, we need to user cross validation.
* **primary_metric**: Area Under Curve (AUC) is one of the most widely used metrics for evaluation. It is used for binary classification problem.
* **task**: Classification, since we want to have a binary prediction (O or 1).

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iterations": 40,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on heart-compute with default configuration
Running on remote compute: heart-compute
Parent Run ID: AutoML_ca508edf-88f5-40cc-b1d0-b10738ed89fb

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing value

## Run Details
Use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_ca508edf-88f5-40cc-b1d0-b10738ed89fb',
 'target': 'heart-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-28T15:39:03.334334Z',
 'endTimeUtc': '2021-01-28T16:01:52.051273Z',
 'properties': {'num_iterations': '40',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'heart-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-heart-experiment","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-136394","workspace_name":"quick-starts-ws-136394","region":"southcentralus","compute_target":"heart-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":40,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model


In [12]:
best_run, fitted_model = remote_run.get_output()

In [14]:
best_metrics = best_run.get_metrics()

print("Best Run Id : {}".format(best_run.id), 
      "Accuracy : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

Best Run Id : AutoML_ca508edf-88f5-40cc-b1d0-b10738ed89fb_36
Accuracy : 0.8628619528619529
Best metrics : {'precision_score_micro': 0.8628619528619529, 'matthews_correlation': 0.6840563395664755, 'recall_score_micro': 0.8628619528619529, 'log_loss': 0.409238819908503, 'AUC_micro': 0.8999717613848928, 'average_precision_score_weighted': 0.8962645546643593, 'recall_score_macro': 0.822949057431816, 'accuracy': 0.8628619528619529, 'average_precision_score_macro': 0.8694571759007962, 'norm_macro_recall': 0.6458981148636321, 'recall_score_weighted': 0.8628619528619529, 'precision_score_macro': 0.8635369260369261, 'f1_score_micro': 0.8628619528619529, 'balanced_accuracy': 0.822949057431816, 'precision_score_weighted': 0.8682143884921661, 'average_precision_score_micro': 0.8994529479805532, 'weighted_accuracy': 0.8943077429194449, 'AUC_macro': 0.8891970788522512, 'f1_score_macro': 0.833392286116287, 'f1_score_weighted': 0.8573981302795405, 'AUC_weighted': 0.8891970788522513, 'confusion_matrix'

In [15]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('11',
                                           Pipeline(memory=None,
                                                    steps=[('robustscaler',
                                                            RobustScaler(copy=True,
                                                                         quantile_range=[25,
                                                                                         75],
                                                                         with_centering=False,
                                                                         with_scaling=False)),
                                                           ('gradientboostingclassifier',
                                                            GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                       criterion='f

In [17]:
# Save the best model
model = best_run.register_model(model_name = "heart-failure-best-model-automl", model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

### Define the environement

In [ ]:
best_run.download_file('outputs-automl/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-env', file_path = 'conda_env.yml')

### Download the scoring file

In [ ]:
best_run.download_file('outputs-automl/scoring_file_v_1_0_0.py', 'score.py')

### Inference Configuration

In [ ]:
inference_config = InferenceConfig(entry_script= 'score.py', environment=environment)

### ACI Configuration

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

### Deploy

In [ ]:
webservice = Model.deploy(workspace=ws, 
                        name='heart-failure-ws', 
                        models=[model], 
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)

In [ ]:
# wait for deployment to finish and display the scoring uri and swagger uri
webservice.wait_for_deployment(show_output=True)

print("Status : {}".format(webservice.state), 
      "scoring_uri : {}".format(webservice.scoring_uri), 
      "swagger_uri : {}".format(webservice.swagger_uri), sep = '\n')

### Consume 
# NOT FINISHED

Send a request to the web service you deployed to test it.

In [ ]:
# select samples from the dataframe
sample = df.sample(4)
target = sample.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = sample.to_dict(orient='records')

data = json.dumps({'data': recored})

# get inference
output = webservice.run(data)
print(output)

Print the logs of the web service and delete the service

In [ ]:
webservice.get_logs()

In [ ]:
webservice.delete()
compute_target.delete()